`__all__`
- 이 모듈(ModernTCN_Layer.py)을 `from ModernTCN_Layer import *` 형태로 import할 때,
외부에 공식적으로 공개되는 심볼들을 지정
- `__all__`이 있으면 지정된 클래스만 불러옴, 없으면 파일 안의 모든 클래스가 import될 수도 있

In [ ]:
#__all__ = ['Transpose', 'get_activation_fn', 'moving_avg', 'series_decomp', 'PositionalEncoding', 'SinCosPosEncoding', 'Coord2dPosEncoding', 'Coord1dPosEncoding', 'positional_encoding']
__all__ = ['moving_avg', 'series_decomp',  'Flatten_Head']
import torch
from torch import nn
import math
# decomposition

## moving_avg: 이동 평균 시계열을 만드는 코드
- 이동 평균 시계열: 원래 시계열에서 일정 구간(윈도우)의 값들을 평균 내서 만든 새로운 시계열
- noise를 줄이고 장기 추세(trend)를 뚜렷하게 보이게 함

입력 x의 모양: (batch, seq_len, channels)

양끝에 패딩을 주고 cat하는 이유
- 시계열 양 끝단에서는 윈도우가 잘려서 평균을 제대로 못 냄
- e.g., 시퀀스가 [x0, x1, x2]와 같고 윈도우 크기가 3일 때, x0 기준 [?, x0, x1] 값을 보고 평균을 구해야 하는데 x0 앞의 값이 없음
- 따라서 양끝을 패딩으로 채워줌

repeat(): 텐서를 복제해서 확장하는 메소드
- x[:, 0:1, :]으로 시계열 맨 앞 timestep만 뽑으면 shape이 (batch, 1, features)가 됨
- .repeat(1, (kernel_size-1)//2, 1)
- 첫 번째 1 → 배치 차원은 반복하지 않음
- 두 번째 (kernel_size-1)//2 → time 차원(timestep)을 이만큼 복제
- 세 번째 1 → feature 차원도 반복하지 않음
- 즉, 맨 앞 timestep을 여러 번 복제해서 새로운 앞쪽 시퀀스를 만듦 (뒷쪽도 마찬가지)

In [ ]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

## series-decomp: 시계열 분해
- moving_avg 블록을 이용해 추세를 계산
- 원본에서 추세를 뺀 나머지 -> res (잔차)
- 잔차와 추세를 반환

잔차-추세 분해를 왜 하는가?
- 추세는 장기적인 변화 양상, 비교적 부드럽고 저주파 성분
- 잔차는 추세에서 벗어난 변동, 노이즈, 비교적 불규칙하고 고주파
- 모델이 한번에 둘을 학습하면 어렵기 때문에, 잔차-추세로 분해해서 문제를 단순화
- 노이즈가 잔차 쪽으로 몰리므로 중요한 패턴(추세)을 더 뚜렷하게 모델에 제공할 수 있음
- 추세는 장기 패턴으로, 잔차는 이상 탐지에 활용 가능

In [ ]:
class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

## Flatten_Head: ModernTCN의 출력 feature map을 task-specific output으로 변환하는 Head

인자
- individual: True면 변수별로 독립적인 head, False면 공통 head
- n_vars: 입력 변수 개수
- nf: 입력 feature 수 (보통 d_model * patch_num)
- target_window: 최종 출력 시퀀스 길이
- head_dropout: head에서 dropout 비율

입력 x의 모양: [batch, nvars, d_model, patch_num]

- flattens: 각 변수마다 feature map을 1D로 펼치는 역할
- linears: 펼친 벡터를 최종 출력(target_window)로 변환
- individual=True면 각 변수별로 flatten, linear, dropout 레이어가 있음

In [ ]:
# forecast task head
class Flatten_Head(nn.Module):
    def __init__(self, individual, n_vars, nf, target_window, head_dropout=0):
        super(Flatten_Head, self).__init__()

        self.individual = individual
        self.n_vars = n_vars

        if self.individual:
            self.linears = nn.ModuleList()
            self.dropouts = nn.ModuleList()
            self.flattens = nn.ModuleList()
            for i in range(self.n_vars):
                self.flattens.append(nn.Flatten(start_dim=-2))
                self.linears.append(nn.Linear(nf, target_window))
                self.dropouts.append(nn.Dropout(head_dropout))
        else:
            self.flatten = nn.Flatten(start_dim=-2)
            self.linear = nn.Linear(nf, target_window)
            self.dropout = nn.Dropout(head_dropout)

    def forward(self, x):  # x: [bs x nvars x d_model x patch_num]
        if self.individual:
            x_out = []
            for i in range(self.n_vars):
                z = self.flattens[i](x[:, i, :, :])  # z: [bs x d_model * patch_num]
                z = self.linears[i](z)  # z: [bs x target_window]
                z = self.dropouts[i](z)
                x_out.append(z)
            x = torch.stack(x_out, dim=1)  # x: [bs x nvars x target_window]
        else:
            x = self.flatten(x)
            x = self.linear(x)
            x = self.dropout(x)
        return x